In [ ]:
DATA_PATH = '../data/'
LIGHTCURVES_PATH = DATA_PATH + 'lightcurves/'
FEATURES_PATH = DATA_PATH + 'features/'

In [ ]:
import numpy as np
import pandas as pd
import measurements, extract
import matplotlib.pyplot as plt

In [ ]:
filename = 'transient_catalog.pickle'
indir = DATA_PATH; filepath = indir + filename
df_cat = pd.read_pickle(filepath)

Import transient lightcurves

In [ ]:
filename = 'transient_lightcurves.pickle'
indir = LIGHTCURVES_PATH; filepath = indir + filename
df_tra = pd.read_pickle(filepath)
df_tra.shape

Filter transient lightcurves

In [ ]:
# Minimum number of observations for each light curve used
min_obs = 10

In [ ]:
# Delete rows of blended observations
df_tra = df_tra.drop_duplicates(['TransientID','MJD'], keep='first')
# Add observation count to every transient
df_count = df_tra.groupby('TransientID', as_index=False).count()
df_count['ObsCount'] = df_count['Mag']
df_count = df_count[['TransientID', 'ObsCount']]
df_tra = df_tra.merge(df_count, how='inner')
# Remove objects with less than min_obs
df_tra = df_tra[df_tra.ObsCount >= min_obs]

Import non-transient light curves

In [ ]:
filename = 'nontransient_lightcurves.pickle'
indir = LIGHTCURVES_PATH; filepath = indir + filename
df_nont = pd.read_pickle(filepath)
df_nont.shape

Filter non-transient lightcurves

In [ ]:
# Delete rows of blended observations
df_nont = df_nont.drop_duplicates(['ID','MJD'], keep='first')
# Add observation count to every nontransient
df_count = df_nont.groupby('ID', as_index=False).count()
df_count['ObsCount'] = df_count['Mag']
df_count = df_count[['ID', 'ObsCount']]
df_nont = df_nont.merge(df_count, how='inner')
# Remove nontransient objects with less than 5 observations
df_nont = df_nont[df_nont.ObsCount >= min_obs]
df_nont.shape

In [ ]:
np.random.seed(42)
# Sample non-transient subset of same size as transients
sample_size = df_tra.TransientID.unique().shape[0]
IDs = np.random.choice(df_nont.ID.unique(), size=sample_size, replace=False)
df_nont = df_nont[df_nont.ID.isin(IDs)]
df_nont.ID.unique().shape, df_nont.shape

Extract transient features

In [ ]:
# Create empty feature dict
tran_feats = extract.feature_dict(31)
for trID in df_tra.TransientID.unique():
    # Get current object light curve
    df = df_tra[df_tra.TransientID == trID]
    # Get features
    obj_feats = extract.features(df, feature_dict)
    # Append features
    for k,v in tran_feats.items():
        if k != 'ID': tran_feats[k].append(obj_feats[k])
    tran_feats['ID'].append(trID)
# Create feature dataframe
df_feat_tran = pd.DataFrame(tran_feats)

Count number of features

In [ ]:
df_feat_tran.count()

Count number of unique features

In [ ]:
df_feat_tran.T.apply(lambda x: x.nunique(), axis=1)

Save transient features

In [ ]:
outdir = FEATURES_PATH

In [ ]:
filename_form = 'transient_{}obs_{}feats.pickle'
# Save all 31 features
num_features = df_feat_tran.shape[1]-1
filename = filename_form.format(min_obs, num_features) 
df_feat_tran.to_pickle(outdir + filename)
# Save all 27 features
df_feat_tran = df_feat_tran.drop(['poly4_a', 'poly4_b', 'poly4_c', 'poly4_d'], axis=1)
num_features = df_feat_tran.shape[1]-1
filename = filename_form.format(min_obs, num_features) 
df_feat_tran.to_pickle(outdir + filename)
# Save all 21 features
df_feat_tran = df_feat_tran.drop(['poly1_a','poly2_a','poly2_b','poly3_a','poly3_b','poly3_c'], axis=1)
num_features = df_feat_tran.shape[1]-1
filename = filename_form.format(min_obs, num_features) 
df_feat_tran.to_pickle(outdir + filename)
# Save all 19 features
df_feat_tran = df_feat_tran.drop(['small_kurtosis','pair_slope_trend_last_30'], axis=1)
num_features = df_feat_tran.shape[1]-1
filename = filename_form.format(min_obs, num_features) 
df_feat_tran.to_pickle(outdir + filename)

Extract nontransient features

In [ ]:
# Create empty feature dict
nontran_feats = feature_dict(31)
for ID in df_nont.ID.unique():
    # Get current object light curve
    df = df_nont[df_nont.ID == ID]
    # Get features
    obj_feats = extract.features(df, feature_dict)
    # Append features
    for k,v in nontran_feats.items():
        if k != 'ID': nontran_feats[k].append(obj_feats[k])
    nontran_feats['ID'].append(ID)
# Create feature dataframe
df_feat_nontran = pd.DataFrame(nontran_feats)

Count number of features

In [ ]:
df_feat_nontran.count()

Count number of unique features

In [ ]:
df_feat_nontran.T.apply(lambda x: x.nunique(), axis=1)

Save nontransient features

In [ ]:
filename_form = 'nontransient_{}obs_{}feats.pickle'
# Save all 31 features
num_features = df_feat_nontran.shape[1]-1
filename = filename_form.format(min_obs, num_features) 
df_feat_nontran.to_pickle(outdir + filename)
# Save all 27 features
df_feat_nontran = df_feat_nontran.drop(['poly4_a', 'poly4_b', 'poly4_c', 'poly4_d'], axis=1)
num_features = df_feat_nontran.shape[1]-1
filename = filename_form.format(min_obs, num_features) 
df_feat_nontran.to_pickle(outdir + filename)
# Save all 21 features
df_feat_nontran = df_feat_nontran.drop(['poly1_a','poly2_a','poly2_b','poly3_a','poly3_b','poly3_c'], axis=1)
num_features = df_feat_nontran.shape[1]-1
filename = filename_form.format(min_obs, num_features) 
df_feat_nontran.to_pickle(outdir + filename)
# Save all 19 features
df_feat_nontran = df_feat_nontran.drop(['small_kurtosis','pair_slope_trend_last_30'], axis=1)
num_features = df_feat_nontran.shape[1]-1
filename = filename_form.format(min_obs, num_features) 
df_feat_nontran.to_pickle(outdir + filename)